In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GlobalAvgPool2D, TimeDistributed, Conv2D, MaxPooling2D, Flatten, Reshape
from tensorflow.keras.applications import MobileNetV2


2025-04-05 17:36:54.938412: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-05 17:36:55.013041: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-05 17:36:55.082411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743889015.152435   59285 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743889015.178307   59285 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743889015.289340   59285 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
def extract_frames(filepath, target_size=(1920, 1080), max_frames=30):
    cap = cv2.VideoCapture(filepath)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, target_size)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame / 255.0
        frames.append(frame)
    
    video_array = np.array(frames)
    cap.release()
    return video_array

In [ ]:
def generate_df(X_folder, y_filepath):
    X_filepaths = []
    X_arrays = []
    for i in range(5):
        X_filename = f"{X_folder}{i+1}.mp4"
        X_filepaths.append(X_filename)
        
    for filepath in X_filepaths:
       X_array = extract_frames(filepath)
       X_arrays.append(X_array)
        
    
    y_df = pd.read_csv(y_filepath)
    y_df.drop(columns='filename', axis=1, inplace=True)
    
    y_array = y_df.to_numpy()
    
    return X_arrays, y_array
        

: 

In [ ]:
X, y = generate_df("../model/data/videos/", "../model/data/coordinates/coordinate_data.csv")

In [ ]:
def create_feature_extraction_cnn(input_shape):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    for layer in base_model.layers:
        layer.trainable = False
        
    model = Sequential([
        base_model,
        GlobalAvgPool2D(),
        Dense(256, activation='relu')
    ])
    
    return model

In [ ]:
def build_model(input_shape, X, y):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape)
    ])
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    model.fit(X, y, epochs=5)
    
    return model

In [ ]:
model = build_model(input_shape=(1920, 1080), X=X, y=y)

2025-04-05 16:48:15.659205: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/itachi/GitHub/Ramblin-Hacks/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 61, 76, 52, 83, 98
'y' sizes: 8
